### Πολλαπλασιασμός

Ας δούμε λίγη assembly!

In [1]:
@code_native 2*5 # Ίδιο αποτέλεσμα C και Julia σε επίπεδο assembly

	.text
Filename: int.jl
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 33
	imulq	%rsi, %rdi
	movq	%rdi, %rax
	popq	%rbp
	retq
	nopl	(%rax)


### Exponentiation

In [2]:
2^5

32

In [3]:
2^(-5) # (Int64 , Int64) -> Int64 (Type Stability)

LoadError: DomainError:
Cannot raise an integer x to a negative power -n. 
Make x a float by adding a zero decimal (e.g. 2.0^-n instead of 2^-n), or write 1/x^n, float(x)^-n, or (x//1)^-n.

Julia μας τα χαλάς...

In [4]:
@code_native ^(2,5)

	.text
Filename: intfuncs.jl
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 197
	callq	power_by_squaring
	popq	%rbp
	retq
	nopl	(%rax,%rax)


Ας φτιάξουμε μια "φυσιολογική" συνάρτηση!

In [5]:
function expo(x,y)
    if y>0
        return x^y
    else
        x = convert(Float64,x)
        return x^y
    end
end

expo (generic function with 1 method)

In [6]:
println(expo(2,5))
println(expo(2,-5))

32
0.03125


In [7]:
@code_native expo(2,5) # Η expo δεν είναι type stable!

	.text
Filename: In[5]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%rbx
	subq	$24, %rsp
	movq	%rdi, %rbx
Source line: 2
	testq	%rdx, %rdx
	jle	L42
Source line: 3
	movabsq	$power_by_squaring, %rax
	movq	%rsi, %rdi
	movq	%rdx, %rsi
	callq	*%rax
	movb	$1, %dl
	movq	%rax, (%rbx)
	jmp	L108
Source line: 5
L42:
	xorps	%xmm0, %xmm0
	cvtsi2sdq	%rsi, %xmm0
Source line: 701
	movsd	%xmm0, -24(%rbp)
	xorps	%xmm1, %xmm1
	cvtsi2sdq	%rdx, %xmm1
Source line: 699
	movsd	%xmm1, -16(%rbp)
	movabsq	$__pow, %rax
	callq	*%rax
	movsd	-24(%rbp), %xmm1        # xmm1 = mem[0],zero
	addsd	-16(%rbp), %xmm1
Source line: 300
	ucomisd	%xmm1, %xmm1
	jp	L102
	ucomisd	%xmm0, %xmm0
	jp	L118
Source line: 6
L102:
	movb	$2, %dl
	movsd	%xmm0, (%rbx)
Source line: 3
L108:
	movq	%rbx, %rax
	addq	$24, %rsp
	popq	%rbx
	popq	%rbp
	retq
Source line: 300
L118:
	movabsq	$jl_throw, %rax
	movabsq	$140365994486944, %rdi  # imm = 0x7FA9813D6CA0
	callq	*%rax
	nopl	(%rax)
